# Task 1 

- Train a GCN on the Citeseer Dataset
- First: Install dependencies. I use the linked dataset to install these dependencies [Wheels](https://www.kaggle.com/datasets/lyomega/torch-geometric)

In [1]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric --no-index --find-links=file:///kaggle/input/torch-geometric

Looking in links: file:///kaggle/input/torch-geometric
Processing /kaggle/input/torch-geometric/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
Processing /kaggle/input/torch-geometric/torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl
Processing /kaggle/input/torch-geometric/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl
Processing /kaggle/input/torch-geometric/torch_spline_conv-1.2.1-cp37-cp37m-linux_x86_64.whl
Processing /kaggle/input/torch-geometric/torch_geometric-2.0.4-py3-none-any.whl


In [2]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.loader import DataLoader
from typing import List, Any
import torchmetrics
import pytorch_lightning as pl

# Load the CiteSeer Dataset

## Dataset Descripiton CiteSeer

The Goal of the CiteSeer Dataset is to classify the category of a paper.  
The Dataset contains:  
- 3327 Papers, which are represented as Nodes in the Dataset. 
- Each Node has a Feature Vector of length 3703.
    - This Feature Vector encodes, if a common Word in the Dataset is present (There are 3703 common words (sum(words) < 10 where dropped))
- There are 9104 edges between Nodes. These Edges represent, if a Paper cite another Paper. 

## Splits

In [3]:
dataset = Planetoid(root="./", name="CiteSeer")
data = dataset[0]

train_mask = data.train_mask ^ data.val_mask# combine train and val set, since we do not have to use the validation set
test_mask = data.test_mask

Processing...
Done!


In [4]:
dataloader = DataLoader(batch_size=1, dataset=dataset)

# Define the Networks and a Lighntning Wrapper for these Networks

In [5]:
class GNNWrapper(pl.LightningModule):

    def __init__(self, 
        loss: callable, 
        lr: float,
        model: torch.nn.Module,
        val_metrics: List[torchmetrics.Metric],
        test_metrics: List[torchmetrics.Metric],
        train_mask,
        test_mask,
    ) -> None:
        super().__init__()

        self.loss = loss
        self.lr = lr
        
        self.model = model
        self.train_mask = train_mask
        self.test_mask = test_mask
        
        # eval stuff
        self.test_metrics = test_metrics
        self.val_metrics = val_metrics

        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.model(x)
        return out

    def _step(self, batch, mask) -> torch.Tensor:
        x, edge_index, y = batch.x, batch.edge_index, batch.y
        x = (x, edge_index)
        pred  = self.forward(x)
        loss = self.loss(pred[mask], y[mask])
        return pred, loss

    def training_step(self, batch, batch_idx) -> torch.Tensor:
        pred, loss = self._step(batch, self.train_mask)
        self.log("train/loss", loss)
        return loss
    
    def _eval_step(self, batch, metrics):
        pred, loss = self._step(batch, self.test_mask)
        for metric in metrics:
            metric.to(self.device)
            metric.update(pred, batch.y)

        return loss

    def _eval_epoch_end(self, eval_type, metrics):
        for metric in metrics:
            value = metric.compute().detach().data
            self.log(f"{eval_type}/{metric.__class__.__name__}:", value)
            print(f"{eval_type} {metric.__class__.__name__}: {value}")
        
    def test_step(self, batch: torch.Tensor, batch_idx: int) -> torch.Tensor:
        loss = self._eval_step(batch, self.test_metrics)

    def test_epoch_end(self, outputs) -> None:
        self._eval_epoch_end("Test", self.test_metrics)

    def validation_step(self, batch: torch.Tensor, batch_idx: int) -> torch.Tensor:
        loss = self._eval_step(batch, self.val_metrics)

    def validation_epoch_end(self, outputs) -> None:
        self._eval_epoch_end("Val",  self.val_metrics)
                
    def configure_optimizers(self) -> Any:
        optim = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optim

class GCN(torch.nn.Module):
    """Simple Convolutional Graph Neural Network"""
    def __init__(self, hidden_dim, node_features, num_classes):
        super().__init__()
        self.conv1 = GCNConv(node_features, hidden_dim)
        self.bn = torch.nn.BatchNorm1d(hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index = data
        x = self.conv1(x, edge_index)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return torch.nn.functional.log_softmax(x, dim=1)

    
class AGN(torch.nn.Module):
    """Attention Based Covolutional Graph Neuralnetwork"""
    def __init__(self, hidden_dim, node_features, num_classes):
        super().__init__()
        self.conv1 = GATConv(node_features, hidden_dim)
        self.conv2 = GATConv(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index = data
        x = self.conv1(x, edge_index)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return torch.nn.functional.log_softmax(x, dim=1)

In [6]:
# Instantiate the GNN's

hidden_dim = 32
node_features = dataset.num_node_features
num_classes = dataset.num_classes
loss = torch.nn.functional.nll_loss
lr = 1e-3
epochs = 500
log_intervall = 10

# Define the models
attention_gnn = AGN(hidden_dim=hidden_dim, node_features=node_features, num_classes=num_classes)
convolutional_gnn = GCN(hidden_dim=hidden_dim, node_features=node_features, num_classes=num_classes)

# Define the metrics
agnn_test_metrics = [torchmetrics.Accuracy(num_classes), torchmetrics.AUROC(num_classes)]
cgnn_test_metrics = [torchmetrics.Accuracy(num_classes), torchmetrics.AUROC(num_classes)]

attention_model = GNNWrapper(
    loss=loss, 
    lr=lr, 
    model=attention_gnn, 
    val_metrics=[], 
    test_metrics=agnn_test_metrics, 
    train_mask=train_mask, 
    test_mask=test_mask
)

convolutional_model = GNNWrapper(
    loss=loss, 
    lr=lr, 
    model=convolutional_gnn, 
    val_metrics=[], 
    test_metrics=cgnn_test_metrics, 
    train_mask=train_mask, 
    test_mask=test_mask
)

/opt/conda/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [7]:
print("Training Attention Model")
trainer = pl.Trainer(max_epochs=epochs, accelerator="gpu", devices=1, log_every_n_steps=log_intervall)
trainer.fit(attention_model, dataloader)

print("Training Convolution Model")
trainer_ = pl.Trainer(max_epochs=epochs, accelerator="gpu", devices=1, log_every_n_steps=log_intervall)
trainer_.fit(convolutional_model, dataloader)

Training Attention Model


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:109: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  category=PossibleUserWarning,
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1896: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Training Convolution Model


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:109: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  category=PossibleUserWarning,
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1896: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

In [8]:
print("Evaluation Attention Model")
trainer.test(attention_model, dataloader)
print("Evaluation Convolution Model")
trainer_.test(convolutional_model, dataloader)

Evaluation Attention Model


Testing: 0it [00:00, ?it/s]

Test Accuracy: 0.7448151707649231
Test AUROC: 0.9083120226860046


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        Test/AUROC:        │    0.9083120226860046     │
│      Test/Accuracy:       │    0.7448151707649231     │
└───────────────────────────┴───────────────────────────┘

Evaluation Convolution Model


Testing: 0it [00:00, ?it/s]

Test Accuracy: 0.754132866859436
Test AUROC: 0.9249188303947449


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        Test/AUROC:        │    0.9249188303947449     │
│      Test/Accuracy:       │     0.754132866859436     │
└───────────────────────────┴───────────────────────────┘

[{'Test/Accuracy:': 0.754132866859436, 'Test/AUROC:': 0.9249188303947449}]